In [30]:
chat = ''
# Enter the path where the chat is stored
file_name = './chat1.txt'
with open(file_name, mode = 'r', encoding = 'utf-8') as f:
    chat = f.readlines()

In [31]:
senders = set()          # To mark whether a message traversed is from a new sender or old
messages = dict()    # Store all messages in structured format
dates = dict()       # Store all dates in structures format
last = None

for msg in chat:
    if not (len(msg) > 12 and msg[2] == '/' and msg[5] == '/' and msg[8] == ',' and (msg[11] == ':' or msg[12] == ':')):
        # This means there was a newline in chat and is a part of previous message
#         print(msg)
#         print(msg[2], msg[5], msg[8], msg[12])
        messages[last][-1] += ' ' + content
        continue
            
    # Extract different components of raw data
    try:
        d, t = msg.index(','), msg.index('-')
        s = t + msg[t:].index(':')
        date = msg[:d]
        tim = msg[d+2:t]
        sender = msg[t+2:s]
        content = msg[s+2:-1]
    except:
        # Irrevelant stuff such as changed group's icon, left group, joined group
        continue
        
    # Adding data
    if sender not in senders:
        senders.add(sender)
        messages[sender] = [content]
        dates[sender] = dict()
    else:
        messages[sender].append(content)
            
    if date in dates[sender]:
        dates[sender][date] += 1
    else:
        dates[sender][date] = 1
            
    last = sender

In [32]:
def sort_dates(dates):
    dates.sort(key = lambda date: list(map(int, date.split('/')))[::-1])
    return dates

In [33]:
def get_date_difference(date1, date2):
    from datetime import date
    d1, m1, y1 = map(int, date1.split('/'))
    d2, m2, y2 = map(int, date2.split('/'))
    diff = date(y2, m2, d2) - date(y1, m1, d1)
    return abs(diff.days)

In [34]:
# Date statistics

def biggest_date_difference(dates):       # Input: list[key list of dates dictionary]
    dates = sort_dates(dates)
    if len(dates) == 1:
        return 0
    max_diff = get_date_difference(dates[0], dates[1])
    date1 = dates[0]
    date2 = dates[1]
    for i in range(1, len(dates)):
        diff = get_date_difference(dates[i-1], dates[i])
        if diff > max_diff:
            max_diff = diff
            date1 = dates[i-1]
            date2 = dates[i]
    
    return max_diff, date1, date2

def longest_streak(dates):                # Input: list[key list of dates dictionary]
    dates = sort_dates(dates)
    streak = 1
    max_streak = 1
    st, max_st = dates[0], dates[0]
    en, max_en = dates[0], dates[0]
    for i in range(1,len(dates)):
        if get_date_difference(dates[i], dates[i-1]) == 1:
            streak += 1
            en = dates[i]
        else:
            if streak > max_streak:
                max_strek = streak
                max_st = st
                max_en = en
            streak = 1
            st = dates[i]
    if streak > max_streak:
        max_strek = streak
        max_st = st
        max_en = en
    return max_streak, max_st, max_en

def most_messages_per_day(dates):              # Input: dictionary[dates dictionary]
    top3 = [[None, 0], [None, 0], [None, 0]]       # [0]: 3rd rank, [1]: 2nd rank [2]: 3rd rank
    for i in dates:
        if top3[0][1] < dates[i]:
            top3[0][0] = i
            top3[0][1] = dates[i]
        top3.sort(key = lambda x: x[1])
    return top3

In [35]:
# Message Counts

def sender_count(sender_msg):
    return len(sender_msg)

In [36]:
# Creating Dashboard
stat_sender = []
total_msgs = 0
print("Sender-wise Statistics:-")
for sender in senders:
    print("********************", sender, "********************")
    
    date_diff, st, en = biggest_date_difference(list(dates[sender].keys()))
    print("Biggest Date Difference:", date_diff, "-> From", st, "to", en)
    print()
    
    streak, st, en = longest_streak(list(dates[sender].keys()))
    print("Longest Streak of messages:", streak, "-> From", st, "to", en)
    print()
    
    top3 = most_messages_per_day(dates[sender])[::-1]
    print("Top 3 most active days:-")
    for day, count in top3:
        print(day, ":", count, "messages")
    print()
    
    count = sender_count(messages[sender])
    print("Total Messages:", count)
    stat_sender.append([sender, count])
    total_msgs += count
    print()
    print()

Sender-wise Statistics:-
******************** Mummy ********************
Biggest Date Difference: 49 -> From 06/01/20 to 24/02/20

Longest Streak of messages: 1 -> From 14/07/19 to 15/07/19

Top 3 most active days:-
28/08/19 : 11 messages
01/01/20 : 6 messages
19/08/19 : 4 messages

Total Messages: 78


******************** Papaji ********************
Biggest Date Difference: 6 -> From 14/03/19 to 20/03/19

Longest Streak of messages: 1 -> From 06/05/20 to 07/05/20

Top 3 most active days:-
30/01/20 : 76 messages
26/01/19 : 48 messages
25/03/20 : 30 messages

Total Messages: 1078


******************** Mrudul ********************
Biggest Date Difference: 12 -> From 09/03/19 to 21/03/19

Longest Streak of messages: 1 -> From 24/04/20 to 25/04/20

Top 3 most active days:-
05/07/19 : 33 messages
24/03/20 : 22 messages
23/11/18 : 17 messages

Total Messages: 827


******************** Namrata Di ********************
Biggest Date Difference: 8 -> From 08/04/19 to 16/04/19

Longest Streak of

In [37]:
# Group Statistics
print("Total Messages:", total_msgs)
stat_sender.sort(key = lambda x: -x[1])
print("Most active participants:-")
ind = 1
for s,c in stat_sender:
    print(str(ind) + ".", s,":", c, "messages --->", str(int((c/total_msgs) * 100))+"% of messages")
    ind += 1

Total Messages: 5048
Most active participants:-
1. Namrata Di : 1232 messages ---> 24% of messages
2. Papaji : 1078 messages ---> 21% of messages
3. Noopur Di : 916 messages ---> 18% of messages
4. Mrudul : 827 messages ---> 16% of messages
5. Jiju : 563 messages ---> 11% of messages
6. Jijaji : 354 messages ---> 7% of messages
7. Mummy : 78 messages ---> 1% of messages


In [38]:
import matplotlib.pylot as plt
labels = []
sizes = []
for l, s in stat_sender:
    labels.append(l)
    sizes.append(s)

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

ModuleNotFoundError: No module named 'matplotlib.pylot'